In [38]:
import psycopg2
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
class trip:
    def __init__(self, init_pos, end_pos, init_time, end_time):
        self.taxi_init_pos = init_pos
        self.taxi_end_pos = end_pos
        self.taxi_init_time = init_time
        self.taxi_end_time = end_time

    def bus_init(self, init_pos, init_stop, init_time, bus_id, trip_id):

        self.bus_init_pos = init_pos
        self.bus_init_stop = init_stop
        self.bus_init_time = init_time

        self.bus_id = bus_id
        self.trip_id = trip_id

    def bud_end(self, end_pos, end_stop, end_time):
        self.bus_end_stop = end_pos
        self.bus_end_pos = end_stop
        self.bus_end_time = end_time
        



In [ ]:
# get taxi initial points
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

initial_points = []
sql = '''
select st_transform(initial_point, 3763)
from taxi_services 
'''
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

for row in results:
    initial_points.append(row[0])
print(initial_points[:5])

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# get the closest bus stop for each initial point
initial_stops = []
for initial_point in initial_points[:5]:
    sql = f'''
    select 
        stop_id,
        st_transform(t.initial_point, 3763) as ip,
        st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763),
        st_transform(t.final_point, 3763),
        t.initial_ts
    from taxi_services t
    cross join stops s
    --where st_transform(t.initial_point, 3763) = '{initial_point}'
    order by st_distance(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763), '{initial_point}')
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    for row in results:
        initial_stops.append((row[0], row[2], row[3], row[4]))
print(initial_stops[:5])

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

stop_ids = []

# todos os autocarros que param nessa initial_stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    select trip_id from stop_times where stop_id = '{initial_stop[0]}'
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    stop_ids.append(results[0])
print(stop_ids[:5])

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# now we get the final stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    with t2 as ( SELECT DISTINCT ON (t.route_id) t.route_id, st.trip_id, st.departure_time, '{datetime.fromtimestamp(initial_stop[3]).time()}', st.stop_id
                    FROM stop_times st
                    JOIN trips t ON st.trip_id = t.trip_id
                    WHERE st.stop_id = '{initial_stop[0]}'
                      AND st.departure_time > '{datetime.fromtimestamp(initial_stop[3]).time()}'
                    ORDER BY t.route_id, extract(epoch from (st.departure_time - '{datetime.fromtimestamp(initial_stop[3]).time()}')))

    select st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}') as p,
    t2.route_id, 
    t2.trip_id,
    s.stop_id
    from t2 
    join stops s on s.stop_id = t2.stop_id
    group by t2.route_id, t2.trip_id, s.stop_id
    order by st_distance(st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}'), '{initial_stop[2]}')
    limit 1
    '''

    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    print(results, "\n")

In [4]:
import psycopg2

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()


# complete query but slow
sql = '''
select 
    t.id,
    s1.stop_id,
    s2.stop_id,
    st_astext(st_transform(t.initial_point, 3763)) as initial_point,
    st_astext(st_transform(t.final_point, 3763)) as final_point,
    st_astext(st_transform(st_setsrid(st_point(s1.stop_lon, s1.stop_lat), 4326), 3763)) as stop_point_init,
    st_astext(st_transform(st_setsrid(st_point(s2.stop_lon, s2.stop_lat), 4326), 3763)) as stop_point_end,
    t.initial_ts,
    t.final_ts,
    s2.departure_time, 
    --no departure time for last stop yet (need a new join this stop_times and that is slow :( )
    s2.trip_id --to later find the final deprature time
from taxi_services t
JOIN LATERAL ( --get init stop_id, s1.stop_id
    SELECT 
        stop_id,
        stop_lon,
        stop_lat
    FROM 
        stops s
    ORDER BY 
        ST_Distance(
            ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763), 
            ST_Transform(t.initial_point, 3763)
        )
    LIMIT 1
) AS s1 ON TRUE
JOIN LATERAL ( --get final stop_id, s2.stop_id
    WITH t2 AS ( --get all trip_ids that pass the init point, only the first trip after intiial_ts from each route will be selected
        SELECT DISTINCT ON (ts.route_id) 
            st.departure_time,
            st.trip_id,
            st.stop_id
        FROM 
            stop_times st
        JOIN 
            trips ts ON st.trip_id = ts.trip_id 
        WHERE 
            st.stop_id = s1.stop_id
            AND 
            st.departure_time > CAST(to_timestamp(t.initial_ts) AS TIME)
        ORDER BY 
            ts.route_id, 
            EXTRACT(EPOCH FROM (st.departure_time - CAST(to_timestamp(t.initial_ts) AS TIME)))
    )
    SELECT --get all point stops that pass on the route
        st.departure_time,
        t2.trip_id,
        --t2.stop_id as init_stop_id,
        st.stop_id,
        s.stop_lat,
        s.stop_lon
    FROM 
        t2 
    JOIN 
        stop_times st ON st.trip_id = t2.trip_id
    JOIN 
        stops s ON s.stop_id = st.stop_id
    GROUP by 
        st.departure_time,
        t2.trip_id, 
        --t2.stop_id,        
        st.stop_id,        
        s.stop_lat,
        s.stop_lon
    ORDER BY 
        ST_Distance(
            ST_ClosestPoint(
                ST_Collect(ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763)),
                ST_Transform(t.final_point, 3763)
            ), 
            ST_Transform(t.final_point, 3763)
        )
    LIMIT 1
) AS s2 ON TRUE
LIMIT 250000;
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
#print(len(results), "\n")
#print(results)

with open('results.txt', 'w+') as f:
    print(results, file=f)


In [23]:
#build a dictionary of the type dic[route_id] = [trip_id] 
trips_in_routes = {}
#build a dictionary of the type dic[trip_id][stop_id] = departure_time
trip_stop_times = {}


sql_get_trips_from_routes = ''' 
SELECT
    route_id, 
    trip_id
FROM trips t;
'''
cursor_psql.execute(sql_get_trips_from_routes)
results = cursor_psql.fetchall()

for r in results:
    if trips_in_routes.get(r[0]) == None:
        trips_in_routes[r[0]] = []
    trips_in_routes[r[0]].append(r[1])

#-------------------------------------------------

sql_get_stop_times = ''' 
    SELECT trip_id, stop_id, departure_time 
    FROM stop_times;
'''

cursor_psql.execute(sql_get_stop_times)
results = cursor_psql.fetchall()

for r in results:
    if trip_stop_times.get(r[0]) == None:
        trip_stop_times[r[0]] = {}
    trip_stop_times[r[0]][r[1]] = r[2]



36986 



In [42]:
#quick query fixed
import psycopg2

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()

sql = ''' 
SELECT
    t.id,
    s.init_stop_id,
    s.end_stop_id,
    st_astext(st_transform(t.initial_point, 3763)) as initial_point,
    st_astext(st_transform(t.final_point, 3763)) as final_point,
    st_astext(st_transform(st_setsrid(st_point(s.init_lon, s.init_lat), 4326), 3763)) as stop_point_init,
    st_astext(st_transform(st_setsrid(st_point(s.end_lon, s.end_lat), 4326), 3763)) as stop_point_end,
    t.initial_ts,
    t.final_ts,
    s.route_id
FROM taxi_services t
JOIN LATERAL (
    WITH p1 AS (
        SELECT 
            stop_id
        FROM 
            stops s
        ORDER BY 
            ST_Distance(
                ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763), 
                ST_Transform(t.initial_point, 3763)
            )
        LIMIT 5
    ),
    p2 AS (
        SELECT DISTINCT ON (rs.route_id, rs.direction_id)
            rs.route_id,
            rs.direction_id,
            rs.stop_sequence,
            p1.stop_id,
            rs.stop_lon,
            rs.stop_lat
        FROM 
            route_stops rs
        JOIN p1 
        ON rs.stop_id = p1.stop_id
        GROUP BY
            rs.route_id,
            rs.direction_id,
            rs.stop_sequence,
            p1.stop_id,
            rs.stop_lon,
            rs.stop_lat
    )
    SELECT
        p2.stop_id as init_stop_id,
        p2.stop_lat as init_lat,
        p2.stop_lon as init_lon,
        rs.stop_id as end_stop_id,
        rs.stop_lat as end_lat,
        rs.stop_lon as end_lon,
        rs.route_id
    FROM
        route_stops rs
    JOIN
        p2
    ON rs.route_id = p2.route_id
    WHERE
        p2.direction_id = rs.direction_id
        AND p2.stop_sequence < rs.stop_sequence
    GROUP BY 
        p2.stop_id,
        p2.stop_lat,
        p2.stop_lon,
        rs.stop_id,
        rs.stop_lat,
        rs.stop_lon,
        rs.route_id
    ORDER BY 
        ST_Distance(
            ST_ClosestPoint(
                ST_Collect(ST_Transform(ST_SetSRID(ST_Point(rs.stop_lon, rs.stop_lat), 4326), 3763)),
                ST_Transform(t.final_point, 3763)
            ), 
            ST_Transform(t.final_point, 3763)
        )
        --ordenar pela soma das distancias
        
    LIMIT 1
) AS s ON TRUE
LIMIT 10000
'''
cursor_psql.execute(sql)
results = []
results = cursor_psql.fetchall()
print(len(results), "\n")
#print(results)

10000 



In [43]:
#for r in results[:5]:
for r in results:
    #print(r)
    t_init = datetime.fromtimestamp(r[7]).time()
    trip = ''
    best_time = -1
    best_time_found = False
    print(r[9])
    for t in trips_in_routes[r[9]]:
        #print(t, r[9], r[1], trip_stop_times.get(t), trip_stop_times[t].get(r[1]))
        if(trip_stop_times.get(t) == None): continue
        if(trip_stop_times[t].get(r[1]) == None): continue
        if(trip_stop_times[t].get(r[2]) == None): continue

        if(trip_stop_times[t][r[1]] > t_init):
            if(best_time_found):
                if trip_stop_times[t][r[1]] < best_time:
                    best_time = trip_stop_times[t][r[1]]
                    trip = t
            else:
                best_time = trip_stop_times[t][r[1]]
                best_time_found = 1
                trip = t
    t_end = -1
    if(trip != ''):
        t_end = trip_stop_times[trip][r[2]]
    print("trip id:", r[0], end = " ")
    print("trip:", trip, end = " ")
    print("init stop:", r[1], end = " ")
    print("end stop:", r[2], end = " ")
    print("init time:", best_time, end = " ")
    print("end time:", t_end)

    

206
trip id: 405669 trip: 206_0_A_6 init stop: DJOA4 end stop: RFZ1 init time: 09:55:26 end time: 09:56:17
901
trip id: 405670 trip: 901_0_Y_30 init stop: LGEQ1 end stop: SBNT init time: 13:03:29 end time: 12:53:00
603
trip id: 405671 trip: 603_0_A_49 init stop: HSJ7 end stop: CVC2 init time: 10:27:00 end time: 10:53:05
302
trip id: 405672 trip: 302_0_A_33 init stop: BVCT1 end stop: PAL3 init time: 13:22:00 end time: 13:17:23
302
trip id: 405673 trip: 302_0_X_22 init stop: PRDJ end stop: SI1 init time: 12:33:00 end time: 13:04:57
303
trip id: 405674 trip: 303_0_U_9 init stop: SBNT2 end stop: SP init time: 12:42:00 end time: 12:19:42
207
trip id: 405675 trip: 207_0_V_34 init stop: CMP1 end stop: PRDJ init time: 13:00:00 end time: 13:13:00
11M
trip id: 405676 trip:  init stop: ESPL1 end stop: DVE2 init time: -1 end time: -1
703
trip id: 405677 trip: 703_1_Y_9 init stop: ACO2 end stop: PRSM2 init time: 13:33:25 end time: 13:45:57
700
trip id: 405678 trip: 700_1_U_64 init stop: CRJ2 end st

In [ ]:
#print(trip_stop_times['503_0_V_47'])
for t in trips_in_routes['703']:
    print(t,len(trip_stop_times[t]))

In [ ]:
st_astext(ST_Transform(ST_SetSRID(ST_Point(stop_lon, stop_lat), 4326), 3763))